In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-02-02 02:28:30--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-02-02 02:28:31 (6.57 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://jbatesy-bucket.s3.us-east-2.amazonaws.com/amazon_reviews_us_Major_Appliances_v1_00.tsv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16199106|R203HPW78Z7N4K|B0067WNSZY|     633038551|FGGF3032MW Galler...|Major Appliances|          5|            0|          0|   N|                Y|If you need a new...|What a great stov...| 2015-08-31|
|         US|   16374060|R2EAIGVLEALSP3|B002QSXK60|     811766671|Best Hand Clothes...|Major Appliances|          5|    

In [6]:
from pyspark.sql.functions import to_date
from pyspark.sql.types import *
# Read in the Review dataset as a DataFrame

In [31]:
Imported_vine_votes20_df = df.filter('total_votes >20')

In [32]:
Imported_vine_votes20_df.count()

5030

In [34]:
temp_df = Imported_vine_votes20_df.withColumn('helpful_votes50',Imported_vine_votes20_df['helpful_votes'] / Imported_vine_votes20_df['total_votes'])
votes50_df = temp_df.filter('helpful_votes50 >= 0.50')
votes50_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|   helpful_votes50|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|         US|   36792525|R3BC75VKJK6LNV|B00RL014P2|       6980707|The Laundry Alter...|Major Appliances|          4|           83|         89|   N|                Y|Warning! Long Rev...|Where do I begin?...| 2015-08-31|0.9325842696629213|
|         US|   13254956| R5XKK92G5N9FU|B00E

In [35]:
votes_count50 = votes50_df.count()
votes_count50

4686

In [36]:
star5_df = votes50_df.filter('star_rating == "5"')
star5_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|   helpful_votes50|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|         US|   50791387| RYMOWYW38WKOB|B00UI18U08|     846535709|LG LMXC23746S 23 ...|Major Appliances|          5|           35|         35|   N|                N|pleasant surprise...|Bought this fridg...| 2015-08-30|               1.0|
|         US|   12120105|R1A4JVOP7N5Q1K|B00X

In [12]:
star5_count = star5_df.count()
star5_count

1981

In [13]:
vine_only_df = votes50_df.filter('vine == "Y"')
vine_only_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|   helpful_votes50|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|         US|   31302915|R20O97P5V9CZ21|B00P7QI4IM|     901651668|Whynter BR-130SB ...|Major Appliances|          3|           21|         27|   Y|                N|Nice but had some...|I was very excite...| 2015-06-07|0.7777777777777778|
|         US|   38320678| RCNB7YGLWKCV5|B00N

In [14]:
vine_only_count= vine_only_df.count()
vine_only_count

35

In [15]:

vine_only_star5 = vine_only_df.filter('star_rating == "5"')
vine_only_star5.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|   helpful_votes50|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|         US|   38320678| RCNB7YGLWKCV5|B00NQFSSWS|     692404913|Samsung Counter T...|Major Appliances|          5|           23|         30|   Y|                N|You wouldn't micr...|I never would hav...| 2015-02-02|0.7666666666666667|
|         US|   53035876| RX1ZUVGBH3HKC|B00N

In [16]:
vine_only_star5_count = vine_only_star5.count()
vine_only_star5_count

18

In [17]:
not_vine_only_df = votes50_df.filter('vine == "N"')
not_vine_only_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|   helpful_votes50|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|         US|   36792525|R3BC75VKJK6LNV|B00RL014P2|       6980707|The Laundry Alter...|Major Appliances|          4|           83|         89|   N|                Y|Warning! Long Rev...|Where do I begin?...| 2015-08-31|0.9325842696629213|
|         US|   13254956| R5XKK92G5N9FU|B00E

In [18]:

not_vine_count = not_vine_only_df.count()
not_vine_count

4957

In [19]:
not_vine_only_5star_df = not_vine_only_df.filter('star_rating == "5"')
not_vine_only_5star_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|   helpful_votes50|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|         US|   50791387| RYMOWYW38WKOB|B00UI18U08|     846535709|LG LMXC23746S 23 ...|Major Appliances|          5|           35|         35|   N|                N|pleasant surprise...|Bought this fridg...| 2015-08-30|               1.0|
|         US|   12120105|R1A4JVOP7N5Q1K|B00X

In [20]:
not_vine_5star_count = not_vine_only_5star_df.count()
not_vine_5star_count

1963

In [21]:
star5_vine_perc_all = (vine_only_star5_count *100/ votes_count50)
star5_vine_perc_all

0.3605769230769231

In [22]:

star5_not_vine_perc_all = (not_vine_5star_count *100/ votes_count50)
star5_not_vine_perc_all

39.322916666666664

In [23]:

star5_vine_perc_all5star = (vine_only_star5_count *100/ star5_count)
star5_vine_perc_all5star

0.9086320040383644

In [24]:
not_vine_5star_perc_all = (not_vine_5star_count*100 / star5_count )
not_vine_5star_perc_all

99.09136799596163

In [25]:

star5vine_all_vine_perc = (vine_only_star5_count *100/ vine_only_count)
star5vine_all_vine_perc

51.42857142857143

In [26]:

star5non_vine_all_nonvine_perc = (not_vine_5star_count *100/ not_vine_count)
star5non_vine_all_nonvine_perc

39.60056485777688

In [27]:

import pandas as pd
summary_counts = {'Count Title': ['Total Filtered Vote Counts','Total Five Star Vote Counts', 'Total Vine Votes Count','Total Vine Five Star Votes Count','Total Non-Vine Votes Count', 'Total Non-Vine Five Star Votes Count'],
                  'Counts': [votes_count50, star5_count, vine_only_count, vine_only_star5_count, not_vine_count,not_vine_5star_count]}
              
summary_counts_df = pd.DataFrame(summary_counts)
summary_counts_df

,Count Title,Counts
0,Total Filtered Vote Counts,4992
1,Total Five Star Vote Counts,1981
2,Total Vine Votes Count,35
3,Total Vine Five Star Votes Count,18
4,Total Non-Vine Votes Count,4957
5,Total Non-Vine Five Star Votes Count,1963


In [28]:

percentages = {'Percent Title' : ['% of 5 Star Vine Only(Paid) to All Filtered Votes', '% of 5 Star Non-Vine Only(Not Paid) to All Filtered Votes', '% of 5 Star Vine Only(Paid) to all 5 Star Votes', '% of 5 Star Non-Vine Only(Not Paid) to All 5 Star Votes', '% of Vine 5 Star to all Vine', '% of Non-Vine 5 star to All non-vine'],
              'Percentage' : [star5_vine_perc_all, star5_not_vine_perc_all, star5_vine_perc_all5star, not_vine_5star_perc_all, star5vine_all_vine_perc, star5non_vine_all_nonvine_perc ]}
percentages_df = pd.DataFrame(percentages)
percentages_df

,Percent Title,Percentage
0,% of 5 Star Vine Only(Paid) to All Filtered Votes,0.360577
1,% of 5 Star Non-Vine Only(Not Paid) to All Fil...,39.322917
2,% of 5 Star Vine Only(Paid) to all 5 Star Votes,0.908632
3,% of 5 Star Non-Vine Only(Not Paid) to All 5 S...,99.091368
4,% of Vine 5 Star to all Vine,51.428571
5,% of Non-Vine 5 star to All non-vine,39.600565
